## AIDI-2005 Capstone II

**Submitted By:** <br>
**Group #3** <br>
Somya Sachan <br>
Shobhit Panwar <br>
Paramvir Singh Bali <br>
Naman Pathak <br>
Yuming Yao

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import yaml
import sqlite3
import time
from spotipy import SpotifyException

Lets create a database for storing all this infomation in it. For this, we'll be using sqlite3.

In [2]:
# Connect to SQLite database
conn = sqlite3.connect("music_recommendation.db")
cursor = conn.cursor()

# Create tracks table in the database
cursor.execute("""
    CREATE TABLE IF NOT EXISTS tracks (
        track_id TEXT PRIMARY KEY,
        track_name TEXT,
        album TEXT,
        album_id TEXT,
        popularity INTEGER,
        duration_ms INTEGER,
        explicit INTEGER,
        artist TEXT,
        artist_id TEXT,
        release_date TEXT,
        danceability REAL,
        energy REAL,
        instrumentalness REAL,
        liveness REAL,
        loudness REAL,
        mode INTEGER,
        speechiness REAL,
        tempo REAL
    )
""")

In [3]:
# Read the YAML file
with open('client_details.yaml', 'r') as file:
    credentials = yaml.safe_load(file)

# Extract the client ID and client secret
client_id = credentials['client_id']
client_secret = credentials['client_secret']

In [4]:
# Set up the client credentials
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# Playlist IDs collected from Spotify
playlist_ids = ['https://open.spotify.com/playlist/6kh8X00EqGovzCJeYyWVTJ', 'https://open.spotify.com/playlist/2npPHC3OnSlCbu4EHDpAbT',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0aBNIs', 'https://open.spotify.com/playlist/37i9dQZF1DXc2aPBXGmXrt',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1SVXaM', 'https://open.spotify.com/playlist/37i9dQZF1DX1PfYnYcpw8w',
                'https://open.spotify.com/playlist/37i9dQZF1DXc7FZ2VBjaeT', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3Jefw4',
                'https://open.spotify.com/playlist/37i9dQZF1DX2oU49YwtXI2', 'https://open.spotify.com/playlist/01gCw02D2b0jcppPEhS5f7',
                'https://open.spotify.com/playlist/37i9dQZF1DX7QOv5kjbU68', 'https://open.spotify.com/playlist/37i9dQZF1DXa0PTjSQ7AeJ',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4oIqZO', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1jSuBO',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0PRpBu', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2LNd6M',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4gTUOY', 'https://open.spotify.com/playlist/37i9dQZF1DWZ8Cy8eCcjXW',
                'https://open.spotify.com/playlist/37i9dQZF1DWZUozJiHy44Y', 'https://open.spotify.com/playlist/37i9dQZF1DXcISkz62UgzG',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1aBeik', 'https://open.spotify.com/playlist/37i9dQZF1DX6bnzK9KPvrz',
                'https://open.spotify.com/playlist/37i9dQZF1DX3fRquEp6m8D', 'https://open.spotify.com/playlist/37i9dQZF1DWWxPM4nWdhyI',
                'https://open.spotify.com/playlist/37i9dQZF1DX55yuR78Invt', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0v3c88',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1i2njq', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3ji9vq',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3x2k4o', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3bDOaf',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4fRiko', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4lBzZ1',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3by276', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2u61Y4',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0jO79m', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1bSHqE',
                'https://open.spotify.com/playlist/37i9dQZF1DXaohnPXGkLv6', 'https://open.spotify.com/playlist/37i9dQZF1DXaQm3ZVg9Z2X',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0684jS', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1ciP4I',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4mwRSU', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2yXXGB',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0rer1m', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1vOtdC',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2QXz5m', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0SmMeI',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO275qeI', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2ZTWSp',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3vuU6c', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3AeAkE',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0PGpEd', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1brcxq',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO45Xg7m', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0JxyUw',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3CT1zW', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO12tsHe',
                'https://open.spotify.com/playlist/37i9dQZF1DWV5wnxQzwPyR', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evNZVVBPG',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4bwDxS', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1kGxAu',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4BaAkp', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2SyYJ7',
                'https://open.spotify.com/playlist/37i9dQZF1DX6cg4h2PoN9y', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3w0wIU',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2HkQXm', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1rYkRG',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2Zd242', 'https://open.spotify.com/playlist/37i9dQZF1DX6p4TJxzMRDe',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evNZY5NHq', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4yAxRC',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0Jy70B', 'https://open.spotify.com/playlist/37i9dQZF1DX2apWzyECwyZ',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO33svt5', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3r3n5S',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4jkBCE', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO43Szeg',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2YsPPW', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3nMr04',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3Adu8w', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1rW8tq',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1Za1Q4', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1IPOOk',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2ZpGiQ', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0vGf4I',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2w0gZW', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3gxo8o',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO25Vnqg', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1KAa52',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2AI1Ta', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0YT088',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0sQwP6', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO00kolG',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1XGbvi', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO197W3C',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0gCG2c', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3oLcOc',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2RkFc4', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2xmY3T',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO476tMI', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1ahqM0',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO34wsE0', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2veHxn',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2abUti', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1EIUtq',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO42EWMo', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2tuUzC',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO21wpvj', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3Ih808',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2O09Hg', 'https://open.spotify.com/playlist/37i9dQZF1DWTDnCxoyoneS',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1bqVuo', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2zXyCI',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3banUl', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2HkQXm',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2S6MNS', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO47cwRq',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3cPMis', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1ahqM0',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2Fo8uc', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2ckaZO',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0yI3tK', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3VkoW4',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4nBpII', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3ZTUmk',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0RCiYg', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4BaAko',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO170LJe', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3wdaYo',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1PXZsc', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3cPMis',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2K9K3U', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO110IyA',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1ND4Gc', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4vD8f6',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0TXLQk', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3TuhDH',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0BEOzm', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3DbIU8',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2OEQHC', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1o6ydy',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO359MM7', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2YsPPW',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1a5BFC', 'https://open.spotify.com/playlist/37i9dQZF1DX1nSIVoqxfC0',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2nSXaa', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0prlgk',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3TbKzu', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1fL2aQ',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1RW5aw', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1fVvft',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2DV7iw', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4nc7dK',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3c8SFq', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3cv9D2',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1ZHiKY', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3R6Tqo',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2l4rrq', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4am2Dm',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1BKoEM', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3JKYkU',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1AR7Pi', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0Wc5ry',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1pZq0g', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO00of28',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO07bvXy', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2TweiY',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3OC4Te', 'https://open.spotify.com/playlist/37i9dQZF1DX08mhnhv6g9b',
                'https://open.spotify.com/playlist/37i9dQZF1DX8kP0ioXjxIA', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO40D0nC',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0X3Z1S', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO00faq4',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2xmY3S', 'https://open.spotify.com/playlist/2MFHsYbIfOb9mkld2K3wp4',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1erDJl', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0izWBq',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3gV2la', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4rvWRa',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1A5PpK', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2Ihqne',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1wFOHS', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2GY1TG',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4fTuIE', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4hsz4o',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2T8208', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2Bx8gr',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3AOBJT', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3pPcxZ',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO4e5iLu', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0Kc0i8',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3j6tfn', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3t7Mpz',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO216tjq', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1HnlL9',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2fgdpx', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3upg4h',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3gAXLR', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2EPupN',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1DHoaY', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2iBPiw',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO1hiJc4', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO25C800',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3Khq6I', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO0h4rYs',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO398ZBC', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3pXaXS',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2MZJmw', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2oBSta',
                'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3s9iaQ', 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO3zuWdi']


song_count = 0 # Used to count the number of songs in our database

max_retries = 5
retry_delay = 2  # Initial delay in seconds

for playlist_id in playlist_ids:
    # Get the playlist tracks
    results = sp.playlist_tracks(playlist_id)
    # Extract relevant track information
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
        
    # Process the track information
    for track in tracks:
        track_id = track["track"]["id"]
        track_name = track["track"]["name"]
        album_name = track["track"]["album"]["name"]
        album_id = track["track"]["album"]["id"]
        popularity = track["track"]["popularity"]
        duration_ms = track["track"]["duration_ms"]
        explicit = track["track"]["explicit"]
        explicit_int = int(explicit)
        artists = [artist["name"] for artist in track["track"]["artists"]]
        artist_str = ", ".join(artists)
        artist_ids = [artist["id"] for artist in track["track"]["artists"]]
        artist_id_str = ", ".join(artist_ids)
        release_date = track["track"]["album"]["release_date"]

        # Retrieve audio features for the track
        audio_features = sp.audio_features(track_id)
        danceability = audio_features[0]["danceability"]
        energy = audio_features[0]["energy"]

        # Check if the track_id already exists in the table
        cursor.execute("SELECT track_id FROM tracks WHERE track_id = ?", (track_id,))
        existing_track = cursor.fetchone()

        if existing_track is None:
            # Track doesn't exist, so perform the INSERT operation
            cursor.execute("""
                INSERT INTO tracks (track_id, track_name, album, album_id, popularity, duration_ms, explicit, artist, artist_id, release_date, danceability, energy)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (track_id, track_name, album_name, album_id, popularity, duration_ms, explicit_int, artist_str, artist_id_str, release_date, danceability, energy))
            conn.commit()
            song_count += 1
        else:
            # Track already exists, you can choose to skip or perform any other desired action
            print(f"Track with track_id {track_id} already exists. Skipping insertion.")
        '''
        print("Track ID:", track_id, type(track_id))
        print("Track Name:", track_name, type(track_name))
        print("Album:", album_name, type(album_name))
        print("Album ID:", album_id, type(album_id))
        print("Popularity:", popularity, type(popularity))
        print("Duration (ms):", duration_ms, type(duration_ms))
        print("Explicit:", explicit_int, type(explicit_int))
        #print("Artists:", ", ".join(artists), type(artists))
        print("Artists:", artist_str, type(artist_str))
        #print("Artist IDs:", ", ".join(artist_ids), type(artist_ids))
        print("Artist IDs:", artist_id_str, type(artist_id_str))
        print("Release Date:", release_date, type(release_date))
        print("Danceability:", danceability, type(danceability))
        print("Energy:", energy, type(energy))
        print("--------------------")
        '''
        # Print the track information    
    print("Total: ", song_count)

In [8]:
#close the database connection
conn.close()

In [1]:
! pip install cookiecutter

In [2]:
cookiecutter -c v1 https://github.com/drivendata/cookiecutter-data-science

SyntaxError: invalid syntax (1997473491.py, line 1)